![](../docs/banner.png)<figcaption style="text-align: right; font-size:9px"><a href="https://pixabay.com/service/license/">Na licencji Pixabay</a></figcaption>

# KONKURS

### Regulamin

1. Konkurs jest realizowany z wykorzystaniem platform Kaggle i Github Classroom
1. Każdy student może posiadać tylko jedno konto użytkownika na platformie Kaggle biorące udział w konkursie.
1. Można przyjąć dowolną nazwę użytkownika za wyjątkiem obraźliwych lub narusząjących dobra osobiste. Nazwa użytkownika biorąca udział w konkursie powinna być wpisana do pliku README.md w przesłanym zadaniu na platformie Github Classroom.
1. Obowiązuje zakaz dzielenia się kodu z pozostałymi uczestnikami konkursu
1. Można wysłać maksymalnie 15 zgłoszeń konkursowych dziennie do ewaluacji
1. Można wybrać tylko jedno zgłoszenie do końcowej oceny
1. Konkurs trwa od 2022.04.27 do 2022.06.09
1. Każda próba oszustwa (manipulacje na zbiorze danych, współdzielenie kodu itp.) będzie wiązała się z usunięciem z uczestinctwa i otrzymaniem 0 pkt za ostatnie zadanie
1. Zgłoszenie jest uznawane za poprawne, jeżeli zajdą wszystkie poniższe punkty:
    - Wysłanie pliku csv z wynikami predykcji na zbiorze testowym na platformę Kaggle
    - Osiągnięcie na tablicy publicznej (public) wyniku przekraczające 0.7 uśrednionej metryki f1-score
    - Zamieszczenie na platformie Github Classroom kodu rozwiązania wraz z wysłanym plikiem submission
1. Za poprawne zgłoszenie konkursowe student otrzyma 13 pkt za zadanie. 
1. Student otrzyma 25 pkt (pełna punktacja) za zgłoszenie, którego wynik ewaluacji na publicznej tablicy wyników (public leaderboard) wyniesie więcej niż 0.8
1. Trójka studentów, która znajdzie się w TOP 3 na prywatnej tablicy wyników, otrzyma 5 punktów dodatkowych do oceny z laboratorium. Osoby te będą poproszone na ostatnich zajęciach o krótkie zaprezentowanie rozwiązania. Nie obowiązuje tutaj podział na grupy, brany jest wynik obejmujący wszystkie grupy laboratoryjne.

### Zgłoszenia rozwiązania 

Pliki zgłoszeniowe powinny mieć dwie kolumny i być w formacie csv: 
- Id - identyfikator przypadku ze zbioru uczącego
- Predicted - Predykowana klasa

Przykładowy plik rozwiązania:

```csv
Id,Predicted
1, 1
2, 0
3, 0
4, 1
5, 1
6, 0
etc.
```

### Metryka 

Zgłoszenie jest rankingowane na podstawie uśrednionej metryki `f1-score` (typ uśrednienia: `micro`)

### Tablica wyników

Podczas realizacji konkursu będą dostępne dwie tablice wyników:
- Publiczna, dostępna od momentu uruchomienia konkursu. Każdy uczestnik będzie mógł podejrzeć aktualne miejsce w rankingu i uzyskany wynik. Wyniki predykcji będą ewaluowane z wykorzystaniem 30% zbioru testowego.
- Prywatna, dostępna po zakończenia konkursu. Każdy uczestnik, będzie mógł podejrzeć aktualne miejsce i uzyskany wynik dopiero po zakończeniu konkursu. Wyniki predykcji będą ewaluowane z wykorzystaniem 70% zbioru testowego.

W związku z tym, że ewaluacja w ramach danej tablicy wyników dokonywana jest na innej próbce danych, miejsce rankingowe może ulec zmianie. 
Podział danych na tablice publiczną i prywatną jest niejawny i ewaluacja wyników następuje po stronie platofrmy Kaggle

## Zbiór danych

Zbiór danych tworzy lista projektów realizowanych z Funduszy Europejskich w Polsce w latach 2014-2020. Zbiór danych został wcześniej oczyszczony i wybrano podzbiór **15** cech. Cecha wyjściowa składa się z dwóch klas i z powodu prowadzenia konkursu została utajniona.

### Podział zbioru

| Podzbiór | Liczba przypadków | Czy podzbiór można modyfikować podzbiór | Czy posiada etykiety | Liczba przypadków klasy 0 | Liczba przypadków klasy 1 | Rozkład klas |
| --- | --- | --- | --- | --- | -- | -- |
| Uczący (train)| 46800 | $\surd$ | $\surd$ | 21857 | 24943 |  0.467 / 0.533 |
| Walidacyjny (dev) | 15600 | $\surd$ | $\surd$ | 7320 | 8280 | 0.469 / 0.531 |
| Testowy (test) | 15600 | $\times$ | $\times$ | - | - | - |



```python
import sys

sys.path.append("../../pdiow-challenge/")

from pdiow.dataset import PDIOWChallengeDatasetLoader
from pandas_profiling import ProfileReport

ds = PDIOWChallengeDatasetLoader("../../pdiow-challenge/dataset/").load()
df = ds.x_train
df["target"] = ds.y_train
```

###  Kolumny w zbiorze danych

Nazwa | Opis |
--- | --- |
title | Tytuł projektu |
description | Skrócony opis do 2000 znaków |
beneficiary_name | Nazwa beneficjenta
fund | Fundusz |
programme |  Program |
eu_financial_rate | Poziom unijnego dofinansowania w procentach w % |
finance_form | Forma finansowania |
territory_type | Typ obszaru, na którym realizowany jest projekt |
start_date | Data rozpoczęcia realizacji projektu
end_date | Data zakończenia realizacji projektu
competitive_procedure | Projekt konkursowy czy pozakonkursowy |
economic_activity_area | Dziedzina działalności gospodarczej, której dotyczy projekt |
project_thematic_objective | Cel projektu
area_of_project_intervention | Obszar wsparcia projektu |
teritorial_delivery_mechanism | Projekt realizowany w ramach terytorialnych mechanizmów wdrażania
y | brak* 


*Opis będzie przedstawiony na ostatnich zajęciach

### Raport z Pandas Profiling zbioru uczącego

```python
report = ProfileReport(df)
report.to_notebook_iframe()
```

## Kod biblioteki pdiow-challenge


```{important}

Wykorzystanie i adaptowanie interfejsów zdefiniowanych w bibliotece jest opcjonalne. Ma ułatwić pracę, niż tworzyć wymóg jej stosowania.

Jedyna funkcjonalność która nie powinna być zmieniona, jest ta dotycząca przygotowania pliku zgłoszeniowego
```

### Pobieranie i wczytywanie zbioru danych

```python
from pdiow.dataset import PDIOWChallengeDatasetLoader

ds = PDIOWChallengeDatasetLoader().load()
```

### Podzbiory danych

W ramach ułatwienia prototypowania modeli można ograniczyć przypadków w zbiorze wykorzystując metodę `subsample()`, która ogranicza rozmiar zbioru do zadanego procentu. 

**Przykład**: Ograniczenie zbioru do 10%

```python
ds.subsample(sample_size=0.1)
```

### Transformacje danych

Przygotowany został szablon, który można wykorzystać w celu łatwego zaadaptowania transformacji danych.

```python
class Transformation(ABC):
    def __init__(self, dataset: PDIOWChallangeDataset):
        self.dataset = dataset
        
    @abstractmethod
    def _fit_transform(self, data: pd.DataFrame) -> pd.DataFrame:
        pass

    @abstractmethod
    def _transform(self, data: pd.DataFrame) -> pd.DataFrame:
        pass

    def transform(self) -> PDIOWChallangeDataset:
        self.dataset.X_train = self._fit_transform(self.dataset.X_train)
        self.dataset.X_dev = self._transform(self.dataset.X_dev)
        self.dataset.X_test = self._transform(self.dataset.X_test)
        return self.dataset
```

Pozwala na zdefiniowanie transformacji, ich definicja i zastosowania na zbiorze uczącym, a w końcowym kroku dokonuje transformacji na zbiorach walidacyjnym i testowym

### Uczenie modelu i ewaluacja

Przygotowany również został szablon do uczenia modeli i jego ewaluacji, który obejmuję implementację metod jak m.in. `fit`, `predict`, `evaluate`. Parametrami Modelu jest zainicjalizowany klasyfikator oraz zbiór danych.

```{imporant}
Przekazany klasyfikator `classifier` musi implementować metody `fit` i `predict`
```

```python
class ClassificationModel:
    def __init__(self, classifier: Any, dataset: PDIOWChallengeDataset) -> None:
        self.classifier = classifier
        self.dataset = dataset
        self._validate_model()

    def _validate_model(self) -> None:
        fit = getattr(self, "fit", None)
        if not callable(fit):
            raise ValueError("Given model has not fit method!")

        predict = getattr(self, "predict", None)
        if not callable(predict):
            raise ValueError("Given model has not predict method!")

    def fit(self) -> None:
        self.classifier.fit(self.dataset.x_train, self.dataset.y_train)

    def predict(self) -> Any:
        return self.classifier.predict(self.dataset.x_test)

    def evaluate(self) -> Any:
        return Evaluator(
            y_true=self.dataset.y_dev,
            y_pred=self.classifier.predict(self.dataset.x_dev),
        ).evaluate()

    def evaluate_cv(self, splits: int = 5) -> Any:
        return CVEvaluator(
            model=self.classifier,
            X=pd.concat(
                [self.dataset.x_train, self.dataset.x_dev],
                axis=0,
                ignore_index=False,
            ),
            Y=pd.concat(
                [self.dataset.y_train, self.dataset.y_dev],
                axis=0,
                ignore_index=False,
            ),
            splits=splits,
        ).evaluate()

    def fit_evaluate(self) -> Any:
        self.fit()
        return self.evaluate()

    def create_submission_file(self, filepath: str) -> None:
        create_submission_file(y_pred=self.predict(), filepath=filepath)
```

Ewaluację modelu możemy przeprowadzić na dwa sposoby:
- standardowy wykorzystując metodę `fit_evaluate()` i zbiór walidacyjny 
- lub metodę `evaluate_cv()`, która łączy zbiory uczący i walidacyjny i dokonuje walidacji krzyżową. 

### Przygotowanie pliku zgłoszeniowego

Do przygotowania pliku zgłoszeniowego można wykorzystać metodę `create_submission_file` w `ClassificationModel`  lub funkcji `create_submission_file` umieszczonej w module `pdiow.submission`.